## Notebook 1: PDF 预处理

在这一系列中，我们将使用开源模型将PDF转换为播客。

获取播客的第一步是找到一个脚本，目前我们的步骤如下：
- 使用任意主题的PDF
- 使用 LLM 模型，将其处理成文本文件。英文用的是Qwen2.5-1.5B，中文使用Qwen2.5-1.5B效果不太好。
- 在下一个 Notebook 中将其改写为播客稿件。

在这个 Notebook 中，我们将上传一个PDF，使用 `PyPDF2` 将其保存为 `.txt` 文件，后续我们会用轻量模型处理文本。

务必记得安装此项目依赖，不然跑不起来～🙂‍↔️

In [10]:
#!pip install -r requirements.txt

这里设置需要处理的PDF文件路径。

另外，如果你想要发挥GPU极致性能，求你更换大参数的模型，虽然轻量级模型对于此任务也能胜任。

In [11]:
pdf_path = '../resources/2402.13116v4.pdf'
DEFAULT_MODEL = "llama3"

In [12]:
import os
import warnings
from typing import Optional

import PyPDF2
from mlx_lm import load, generate
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

这里会检查文件有没有什么毛病～

In [13]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: 此文件找不到啊🤔: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: 此文件不是 PDF 😵‍💫")
        return False
    return True

这里就简单的读取 PDF 保存为`.txt`文件。默认的最大字符数是10万，如果你有长篇大论要处理，随时可以调整哦！不过，记得要考虑模型的长度限制～

In [14]:
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):
        return None

    try:
        with open(file_path, 'rb') as file:
            # 创建 PDF reader 对象
            pdf_reader = PyPDF2.PdfReader(file)

            # 获取总页数
            num_pages = len(pdf_reader.pages)
            print(f"处理 {num_pages} 页的 PDF...")

            extracted_text = []
            total_chars = 0

            # 遍历所有页面
            for page_num in range(num_pages):
                # 提取当前页面的文本
                page = pdf_reader.pages[page_num]
                text = page.extract_text()

                # 检查文本是否超过最大字符数
                if total_chars + len(text) > max_chars:
                    # 仅截取到最大字符数
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"在第 {page_num + 1} 页超过 {max_chars} 字符限制啦！！")
                    break

                extracted_text.append(text)
                total_chars += len(text)
                print(f"已处理 {page_num + 1}/{num_pages} 页")

            final_text = '\n'.join(extracted_text)
            print(f"\n提取完成！总字数：{len(final_text)}")
            return final_text

    except PyPDF2.PdfReadError:
        print("Error: 这 PDF 有毛病啊，请仔细看看～")
        return None
    except Exception as e:
        print(f"完了，挂了: {str(e)}")
        return None


获取 PDF 元信息

In [15]:
# 获取 PDF 元信息
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):
        return None

    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata
            }
            return metadata
    except Exception as e:
        print(f"提取元信息失败: {str(e)}")
        return None

Finally, we can run our logic to extract the details from the file

In [ ]:
# 提取元信息
print("正在提取元信息...")
metadata = get_pdf_metadata(pdf_path)
if metadata:
    print("\nPDF 元信息:")
    print(f"页数: {metadata['num_pages']}")
    print("文档信息:")
    for key, value in metadata['metadata'].items():
        print(f"{key}: {value}")

# 提取文本
print("\n提取文本...")
extracted_text = extract_text_from_pdf(pdf_path)

# 预览前500字符文本
if extracted_text:
    print("\n预览提取后的文本 (前500字符):")
    print("-" * 50)
    print(extracted_text[:500])
    print("-" * 50)
    print(f"\n提取的总字符数: {len(extracted_text)}")

# 将提取的文本保存到 txt 文件中
if extracted_text:
    output_file = './resources/extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"\n提取的文本已保存到 {output_file}")

### 预处理
现在我们来说说为啥不用正则表达式的提取，而使用大型语言模型的：
目前，我们已经从 PDF 中提取出一个文本文件。一般来说，由于字符、格式、Latex、表格等原因，PDF 提取出的内容可能会很凌乱。
解决这个问题的一种方法是使用正则表达式，但我们也可以使用轻量模型帮助清洗文本。
你可以试试修改 `SYS_PROMPT`

In [16]:
SYS_PROMPT = """您是一流的文本预处理器。以下是来自 PDF 的原始数据，请解析并以清晰可用的方式返回给播客作者。
原始数据中包含换行符、Latex 数学公式以及一些可以完全删除的冗余内容。请去掉对播客作者记录无用的信息。
不论主题如何，以上问题并不详尽，因此请聪明选择要删除的信息，发挥创造力。
请注意，您的任务仅限于清理文本和必要时重写，在删除细节时要果断。当您进行文本处理时，请确保输出中文。
如果遇到英文或其他语言，请翻译为中文后再输出。
请勿添加 Markdown 格式化或特殊字符。
一开始便直接开始响应处理后的文本，不做任何确认，谢谢！
这里是文本：
"""

为了避免模型一次处理整个文件，我们将分块处理文件。

In [17]:
def create_word_bounded_chunks(text, target_chunk_size):
    """
    这里会根据 target_chunk_size 分块
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1  # +1 for the space
        if current_length + word_length > target_chunk_size and current_chunk:
            # Join the current chunk and add it to chunks
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

让我们开始处理文本块吧！✨

In [2]:
import openai
openapi_client = openai.Client(base_url="http://localhost:8080/v1",api_key='NA')

def process_chunk_with_webapi(text_chunk, chunk_num):
    """处理文本块，并返回模型处理好的文本"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    response = openapi_client.chat.completions.create(
        messages=conversation,
        model=DEFAULT_MODEL
    )
    processed_text = response.choices[0].message.content

    # Print chunk information for monitoring
    #print(f"\n{'='*40} Chunk {chunk_num} {'='*40}")
    print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 chars of input
    print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 chars of output
    print(f"{'=' * 90}\n")

    return processed_text

In [20]:
INPUT_FILE = "./resources/extracted_text.txt"  
CHUNK_SIZE = 1000

# 读取文件
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

chunks = create_word_bounded_chunks(extracted_text, CHUNK_SIZE)
num_chunks = len(chunks)

In [21]:
num_chunks

101

In [22]:
# 创建输出文件路径
output_file = f"./resources/clean_{os.path.basename(INPUT_FILE)}"

processed_text = ""

In [ ]:
with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):
        # 处理文本块
        processed_chunk = process_chunk_with_webapi(chunk, chunk_num)
        processed_text += processed_chunk + "\n"

        # 将处理后的文本写入输出文件
        out_file.write(processed_chunk + "\n")
        out_file.flush()

让我们看看最后处理的效果吧～🍻

In [ ]:
print(f"\n处理完成！")
print(f"输入文件: {INPUT_FILE}")
print(f"输出文件: {output_file}")
print(f"已处理总文本块: {num_chunks}")

# 预览处理后的文本的开头和结尾。
print("\n预览处理后的文本：")
print("\n开头:")
print(processed_text[:1000])
print("\n...\n\n结尾:")
print(processed_text[-1000:])

### 下一个 Notebook: 转录员

现在我们已经预处理好文本，在下一个 Notebook 中将其转换为讲稿

In [32]:
#fin